
                                                                          Abstract 
<sub>
This paper aims to address the classic Stigler Diet problem by incorporating up-to-date supermarket
data and nutritional information from the US Department of Agriculture. The study begins by scraping
food prices and food names from dynamic supermarket websites by using Python libraries Selenium,
and BeautifulSoup. To associate the scraped food items with their corresponding nutritional values,
the Levenshtein distance is employed as a similarity measure between food names corresponding to
nutritional values found in the US Department of Agriculture. The primal-dual path-following method,
an interior-point algorithm for solving linear programming problems, is utilized to find the optimal Stigler
Diet.
</sub>
 

&nbsp;
&nbsp;

# 1. Introduction:
 
 The Stigler Diet problem is a classic optimization problem in linear programming, proposed by George Stigler in 1945 for whcih he later won the Nobel Prize. The goal of the problem is to find the least expensive combination of foods that meets a given set of nutritional requirements. Stigler used data on food prices and nutritional content to identify a diet that would minimize the cost while satisfying the recommended daily allowances (RDAs) of various nutrients. However, it has the following limitations:


* Individual nutrient requirements can vary based on factors such as age, gender, activity level, and health status.
* It focuses solely on minimizing cost, disregarding factors such as taste, cultural preferences, and individual food aversions (like food allergies).
* It does not account for the importance of variety and a balanced diet.
* Food prices, availability, and nutritional content may change over time.
* It misses other essential nutrients or non-nutrient factors that contribute to a healthy diet.

### Case Study Safeway: Dynamic website data scraped with Selenium and Beautiful Soup

Using https://www.safeway.com/, it was quickly realized that this is a dynamic website, meaning just requesting it and using BeautifulSoup would not be enough. Selenium was imported to deal with dynamic websites.

In [7]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re


# Set display options to show all rows and columns (it is going to be useful later on for data cleaning)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)


In [14]:

# THIS WAS OK IN JUST SOME ITEMS, BETTER WAYS FOUND
# Set up Selenium webdriver
driver = webdriver.Chrome()

#GETTING MEAT-SEAFOOD DATA
driver.get("https://www.safeway.com/shop/aisles/meat-seafood.3132.html")

# Wait for the dynamically loaded content to load
time.sleep(5)

# Use Selenium to scroll down the page and load more products
for i in range(3):
    driver.execute_script("window.scrollBy(0, 500);")
    time.sleep(2)

# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
meat_seafood = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

In [2]:
# Find all products on the page
products = meat_seafood.find_all('div', {'class': 'product-title'}, {'data-name'})


meat_names = []
meat_prices = []
# Extract the name and price of each product
for product in products:
    name = product.a.get('aria-label')
    price = product.find('div', {'class': 'product-title__qty'}).text
    meat_names.append(name)
    meat_prices.append(price)
# print(meat_names, meat_prices)

df_meat_seafood = pd.DataFrame({'Meat Names': meat_names, 'Meat Prices': meat_prices})
df_meat_seafood


While the previous method works, it is limited in its scope for two reasons. That specific page has releases
of the most popular items. Another approach was used where all the items of specific food types were kept.
However, this website had to be manually loaded. A script to automatically ”click” to load more was used
but it failed. Ultimately, Selenium allows for manually telling it when to fetch the data. The product’s name
and respective price were found with:

In [36]:
# GETTING ALL MEAT AND SEAFOOD

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/search-results.html?q=meat")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_meat_seafood = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [3]:
# Find all MEAT AND SEAFOOD products on the page

products_names = all_meat_seafood.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_meat_seafood.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_meat_seafood_names = [name.text for name in products_names]
all_meat_seafood_prices = [name.text for name in products_prices]


df_all_meat_seafood = pd.DataFrame({'All Meat & Seafood Names': all_meat_seafood_names, 'All Meat & Seafood Prices': all_meat_seafood_prices})
df_all_meat_seafood.to_csv('all_meat_seafood.csv', index=False)

In [8]:
data = pd.read_csv('all_meat_seafood.csv')
print(data.head())

                            All Meat & Seafood Names All Meat & Seafood Prices
0   Ground Beef 80% Lean 20% Fat Value Pack - 3 Lbs.              ($4.99 / Lb)
1        Lauras Ground Beef 85% Lean 15% Fat - 1 Lb.              ($7.99 / Lb)
2              USDA Choice Beef For Stew - 1.50 Lbs.              ($7.99 / Lb)
3  Signature Farms Bone In Chicken Thighs Value P...              ($2.49 / Lb)
4  USDA Choice Beef Tenderloin Filet Mignon Steak...             ($23.99 / Lb)


In [40]:
#GETTING Yogurt & Pudding

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/dairy-eggs-cheese/yogurt-pudding.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_yogurt_pudding = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [10]:
# Find all Yogurt & Pudding products on the page

products_names = all_yogurt_pudding.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_yogurt_pudding.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_yogurt_pudding_names = [name.text for name in products_names]
all_yogurt_pudding_prices = [name.text for name in products_prices]


df_all_yogurt_pudding = pd.DataFrame({'All Yogurt & Pudding Names': all_yogurt_pudding_names, 'All Yogurt & Pudding Prices': all_yogurt_pudding_prices})
df_all_yogurt_pudding.to_csv('all_yogurt_pudding.csv', index=False)

In [9]:
data = pd.read_csv('all_yogurt_pudding.csv')
print(data.head())

                          All Yogurt & Pudding Names  \
0  Yoplait Original Yogurt Low Fat Strawberry Che...   
1                      Zoi Greek Lemon Cream - 32 Oz   
2   FAGE Total 0% Milkfat Plain Greek Yogurt - 32 Oz   
3  Oikos Triple Zero Mixed Berry... ek Yogurt - 4...   
4  Yoplait Original Yogurt Low Fa... rvest Peach ...   

  All Yogurt & Pudding Prices  
0             ($0.16 / Ounce)  
1             ($0.17 / Ounce)  
2             ($0.22 / Ounce)  
3             ($0.26 / Ounce)  
4             ($0.15 / Ounce)  


In [76]:
#GETTING Cheese

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/dairy-eggs-cheese/cheese.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_cheese = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [12]:
# Find all Cheese products on the page

products_names = all_cheese.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_cheese.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_cheese_names = [name.text for name in products_names]
all_cheese_prices = [name.text for name in products_prices]


df_all_cheese = pd.DataFrame({'All Cheese Names': all_cheese_names, 'All Cheese Prices': all_cheese_prices})
df_all_cheese.to_csv('all_cheese.csv', index=False)

In [11]:
data = pd.read_csv('all_cheese.csv')
print(data.head())

                                    All Cheese Names All Cheese Prices
0      Lucerne Cheese Natural Medium Cheddar - 32 Oz   ($0.31 / Ounce)
1  Frigo Cheese Heads String Chee... ck 8 Count -...   ($0.55 / Ounce)
2               Lucerne Cheese Sharp Cheddar - 32 Oz   ($0.31 / Ounce)
3  Frigo Cheese Heads Cheese String 24 Count - 24 Oz   ($0.36 / Ounce)
4          Lucerne Cheese Natural Colby Jack - 32 Oz   ($0.31 / Ounce)


In [46]:
#GETTING MILK & CREAM

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/dairy-eggs-cheese/milk-cream.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_milk_cream = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [13]:
# Find all MILK & CREAM products on the page

products_names = all_milk_cream.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_milk_cream.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_milk_cream_names = [name.text for name in products_names]
all_milk_cream_prices = [name.text for name in products_prices]


df_all_milk_cream = pd.DataFrame({'All Cheese Names': all_milk_cream_names, 'All Cheese Prices': all_milk_cream_prices})
df_all_milk_cream.to_csv('all_milk_cream.csv', index=False)

In [14]:
data = pd.read_csv('all_milk_cream.csv')
print(data.head())

                                    All Cheese Names All Cheese Prices
0          Lucerne Milk Whole 1 Gallon - 128 Fl. Oz.   ($0.05 / Fl.oz)
1    Lucerne Milk - Half Gallon (container may vary)   ($0.06 / Fl.oz)
2          Lucerne Heavy Whipping Cream - 32 Fl. Oz.   ($0.24 / Fl.oz)
3                Lactaid 2% Reduced Fat Milk - 96 Oz   ($0.08 / Fl.oz)
4  O Organics Organic Whole Milk with Vitamin D -...   ($0.06 / Fl.oz)


In [51]:
#GETTING BUTTER & SOUR CREAM

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/dairy-eggs-cheese/butter-sour-cream.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_butter_sour_cream = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [15]:
# Find all BUTTER & SOUR CREAM products on the page

products_names = all_butter_sour_cream.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_butter_sour_cream.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_butter_sour_cream_names = [name.text for name in products_names]
all_butter_sour_cream_prices = [name.text for name in products_prices]


df_all_butter_sour_cream = pd.DataFrame({'All Butter & Sour Cream Names': all_butter_sour_cream_names, 'All Butter & Sour Cream Prices': all_butter_sour_cream_prices})
df_all_butter_sour_cream.to_csv('all_butter_sour_cream.csv', index=False)

In [17]:
data = pd.read_csv('all_butter_sour_cream.csv')
print(data.head())

                       All Butter & Sour Cream Names  \
0  Lucerne Butter Salted Sweet Cream 4 Quarters -...   
1       Earth Balance Vegan Buttery Sticks - 4-16 Oz   
2                    Challenge Butter Salted - 16 Oz   
3     Earth Balance Olive Oil Buttery Spread - 13 Oz   
4  Lucerne Unsalted Sweet Cream Butter Quarters -...   

  All Butter & Sour Cream Prices  
0                ($0.37 / Ounce)  
1                ($0.44 / Ounce)  
2                ($0.37 / Ounce)  
3                ($0.54 / Ounce)  
4                ($0.37 / Ounce)  


In [53]:
#GETTING EGGS

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/dairy-eggs-cheese/eggs.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_eggs = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [18]:
# Find all EGGS products on the page

products_names = all_eggs.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_eggs.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_eggs_names = [name.text for name in products_names]
all_eggs_prices = [name.text for name in products_prices]


df_all_eggs = pd.DataFrame({'All Eggs Names': all_eggs_names, 'All Eggs Prices': all_eggs_prices})
df_all_eggs
df_all_eggs.to_csv('all_eggs.csv', index=False)

In [19]:
data = pd.read_csv('all_eggs.csv')
print(data.head())

                                      All Eggs Names  All Eggs Prices
0      Lucerne Farms Eggs Cage Free Large - 18 Count  ($4.66 / Dozen)
1  Vital Farms Pasture-Raised Eggs - Large - 18 C...  ($7.33 / Dozen)
2      Lucerne Farms Eggs Large Cage Free - 12 Count  ($4.89 / Dozen)
3                  Vital Farms Eggs Large - 12 Count  ($8.99 / Dozen)
4     O Organics Organic Eggs Large Brown - 12 Count  ($5.99 / Dozen)


In [55]:
#GETTING RICE

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/grains-pasta-sides/rice.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_rice = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [20]:
# Find all RICE products on the page

products_names = all_rice.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_rice.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_rice_names = [name.text for name in products_names]
all_rice_prices = [name.text for name in products_prices]


df_all_rice = pd.DataFrame({'All Rice Names': all_rice_names, 'All Rice Prices': all_rice_prices})
df_all_rice.to_csv('all_rice.csv', index=False)

In [21]:
data = pd.read_csv('all_rice.csv')
print(data.head())

                                      All Rice Names  All Rice Prices
0   Signature SELECT Rice Enriched Long Grain - 5 Lb     ($1.10 / Lb)
1                       Mahatma Rice Jasmine - 80 Oz     ($1.80 / Lb)
2  Signature SELECT Rice Thai Jasmine Long Grain ...     ($2.25 / Lb)
3  Signature SELECT Rice Enriched Long Grain - 32 Oz  ($0.08 / Ounce)
4  Ben's Original Parboiled Long... Dry Rice Box ...     ($2.75 / Lb)


In [57]:
#GETTING STUFFING

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/grains-pasta-sides/stuffing-mix.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_stuffing = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [22]:
# Find all STUFFING products on the page

products_names = all_stuffing.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_stuffing.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_stuffing_names = [name.text for name in products_names]
all_stuffing_prices = [name.text for name in products_prices]


df_all_stuffing = pd.DataFrame({'All Stuffing Names': all_stuffing_names, 'All Stuffing Prices': all_stuffing_prices})
df_all_stuffing.to_csv('all_stuffing.csv', index=False)

In [23]:
data = pd.read_csv('all_stuffing.csv')
print(data.head())

                                  All Stuffing Names All Stuffing Prices
0     Stove Top Savory Herbs Stuffing Mix Box - 6 Oz     ($0.63 / Ounce)
1  Signature SELECT Stuffing Mix Chicken Flavored...     ($0.33 / Ounce)
2      Stove Top Stuffing Mix for Chicken Box - 6 Oz     ($0.63 / Ounce)
3        Stove Top Cornbread Stuffing Mix Box - 6 Oz     ($0.63 / Ounce)
4       Stove Top Stuffing Mix for Turkey Box - 6 Oz     ($0.63 / Ounce)


In [59]:
#GETTING GRAINS

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/grains-pasta-sides/grains.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_grains = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [60]:
# Find all GRAINS products on the page

products_names = all_grains.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_grains.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_grains_names = [name.text for name in products_names]
all_grains_prices = [name.text for name in products_prices]


df_all_grains = pd.DataFrame({'All Grains Names': all_grains_names, 'All Grains Prices': all_grains_prices})
df_all_grains

,All Grains Names,All Grains Prices
0,O Organics Organic Quinoa - 16 Oz,($0.50 / Ounce)
1,Signature SELECT Beans Pearl Barley Dry - 16 Oz,($0.14 / Ounce)
2,O Organics Organic Quinoa Quick-Cook - 8 Oz,($0.62 / Ounce)
3,O Organics Organic Quinoa - 64 Oz,($6.25 / Lb)
4,Minute Ready to Serve! Rice Mi... Quinoa Cup -...,($0.36 / Ounce)
5,Quinoa Cooked - 17.6 Oz,($0.34 / Ounce)
6,Minute Multi-Grain Medley - 4-3 Oz,($0.37 / Ounce)
7,Arrowhead Mills Organic Barley Pearled - 28 Oz,($0.27 / Ounce)
8,Bob's Red Mill Organic Whole G... Free Quinoa ...,($0.50 / Ounce)
9,Arrowhead Mills Organic Quinoa - 14 Oz,($0.68 / Ounce)


In [91]:
df_all_grains.to_csv('all_grains.csv', index=False)

In [61]:
#GETTING PASTA

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/grains-pasta-sides/pasta.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_pasta = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [24]:
# Find all PASTA products on the page

products_names = all_pasta.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_pasta.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_pasta_names = [name.text for name in products_names]
all_pasta_prices = [name.text for name in products_prices]


df_all_pasta = pd.DataFrame({'All Pasta Names': all_pasta_names, 'All Pasta Prices': all_pasta_prices})
df_all_pasta.to_csv('all_pasta.csv', index=False)

In [25]:
data = pd.read_csv('all_pasta.csv')
print(data.head())

                                     All Pasta Names All Pasta Prices
0  Signature SELECT Pasta Egg Noodles Wide Bag - ...  ($0.31 / Ounce)
1  Ancient Harvest Supergrain Pas... Spaghetti Bo...  ($0.56 / Ounce)
2    Barilla Pasta Spaghetti Gluten Free Box - 12 Oz  ($0.21 / Ounce)
3       Signature SELECT Pasta Spaghetti Box - 16 Oz  ($0.10 / Ounce)
4       Signature SELECT Pasta Spaghetti Box - 32 Oz  ($0.11 / Ounce)


In [8]:
# GETTING ALL FRUITS

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/fruits-vegetables/fresh-fruits.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_fruits = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


In [26]:
# Find all FRUITS products on the page

products_names = all_fruits.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_fruits.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_fruits_names = [name.text for name in products_names]
all_fruits_prices = [name.text for name in products_prices]


df_all_fruits = pd.DataFrame({'All Fruits Names': all_fruits_names, 'All Fruits Prices': all_fruits_prices})
df_all_fruits.to_csv('all_fruits.csv', index=False)

In [27]:
data = pd.read_csv('all_fruits.csv')
print(data.head())

                        All Fruits Names All Fruits Prices
0                          Banana - Each    ($0.33 / Each)
1          Strawberries Prepacked - 1 Lb      ($4.99 / Lb)
2           Green Seedless Grapes - 2 Lb      ($3.49 / Lb)
3          Blueberries Prepacked - 18 Oz   ($0.39 / Ounce)
4  Organic Strawberries Prepacked - 1 Lb      ($5.99 / Lb)


In [10]:
# GETTING ALL VEGGIES

driver = webdriver.Chrome()
driver.get("https://www.safeway.com/shop/aisles/fruits-vegetables/fresh-vegetables-herbs.3132.html")

# Manually scroll down the page to the desired location
input("Scroll down to the desired location on the page, then press Enter to continue...")


# Get the HTML of the page and parse it with BeautifulSoup
# KEEP THESE DATA SAVED
html = driver.page_source
all_veggies = BeautifulSoup(html, 'html.parser')

# Close the browser window
driver.quit()

Scroll down to the desired location on the page, then press Enter to continue... 


Subsequently, the same was done for all food types. Additional problems occurred when getting vegetables, as some of the pricing information was missing

In [28]:
# Find all VEGGIES products on the page

# products = all_vegies_fruits.find_all('div', {'class': 'product-title'}, {'data-name'})


products_names = all_veggies.find_all('a', {'data-qa': 'prd-itm-pttl'}, {'class': 'product-title__name'})
products_prices = all_veggies.find_all('div', {'data-qa': 'prd-itm-pprc-qty'}, {'class': 'product-title__name'})

all_veggies_names = [name.text for name in products_names]
all_veggies_prices = [name.text for name in products_prices]


# data cleaning via looking for missing price values (see below for the values found and cleaned out:

# list1 = all_veggies_names
# list2 = all_veggies_prices

# if len(list1) < len(list2):
#     list1 += [float('nan')] * (len(list2) - len(list1))
# else:
#     list2 += [float('nan')] * (len(list1) - len(list2))

# df = pd.DataFrame({'col1': list1, 'col2': list2})

# # Set display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# print(df)

# cleaning addittional data for foods that have fuzzy price values
del all_veggies_names[69]
del all_veggies_names[157]
del all_veggies_names[229]
del all_veggies_names[293]
del all_veggies_names[346]
del all_veggies_names[363]
del all_veggies_names[411]
del all_veggies_names[431]


df_all_veggies = pd.DataFrame({'All Veggies Names': all_veggies_names, 'All Veggies Prices': all_veggies_prices})
df_all_veggies.to_csv('all_veggies.csv', index=False)

In [29]:
data = pd.read_csv('all_veggies.csv')
print(data.head())

                                   All Veggies Names All Veggies Prices
0                                           Cucumber     ($1.00 / Each)
1                                     Broccoli Crown       ($3.49 / Lb)
2                                    Green Asparagus       ($4.99 / Lb)
3  Signature Farms Romaine Hearts Prepackaged - 3...    ($1.33 / Count)
4                                   Celery - 1 Bunch     ($2.99 / Each)


The data was subsequently converted into several CSV files and concatenated into a large CSV file, where
the price per mass was calculated

In [24]:
# List of CSV files to concatenate
csv_files = ['all_meat_seafood.csv', 'all_yogurt_pudding.csv', 'all_cheese.csv', 'all_milk_cream.csv', 'all_butter_sour_cream.csv', 'all_eggs.csv', 'all_rice.csv', 'all_grains.csv', 'all_stuffing.csv', 'all_pasta.csv', 'all_fruits.csv', 'all_veggies.csv']

# List to store DataFrames for each CSV file
dfs = []

# Loop through CSV files and read them into a DataFrame, removing headers
for csv_file in csv_files:
    df = pd.read_csv(csv_file, usecols=[0, 1], skiprows=[0], header=None)
    df.columns = ['Name', 'Price']
    dfs.append(df)

# Concatenate DataFrames into a single DataFrame
result = pd.concat(dfs)

# Save concatenated DataFrame to a new CSV file
result.to_csv('concatenated_file.csv', index=False)

In [51]:
# read in the csv file
df = pd.read_csv('concatenated_file.csv')

# function to extract float from string
def extract_float(text):
    try:
        return float(re.findall('\d+\.\d+|\d+', text)[0])
    except:
        return 0

# create new column by multiplying values from columns 1 and 2
df['Mass per money'] = df['Name'].apply(lambda x: extract_float(x.split("-")[-1])).astype(float) * df['Price'].apply(extract_float).astype(float)

# create fourth column based on 'Mass per money' column criteria
df['Fourth Column'] = df['Mass per money']
df.loc[df['Mass per money'] == 0, 'Fourth Column'] = df['Price'].apply(extract_float).astype(float)

# save updated dataframe to a new csv file
df.to_csv('UPDATED_ALL_DATA.csv', index=False)


In [53]:
# read in the updated csv file
df = pd.read_csv('UPDATED_ALL_DATA.csv')

# create new dataframe with only first and fourth columns
new_df = df.loc[:, ['Name', 'Fourth Column']]

# save new dataframe to a new csv file
new_df.to_csv('ALL_FOOD_PRICE.csv', index=False)

Unfortunately, the Safeway nutritional data has to be clicked on, one by one, to be obtained, and there are
over 5k values! There is an alternative solution. The US Department of Agriculture has a data dump file
”2019-2020 FNDDS At A Glance - FNDDS Nutrient Values” in https://www.usda.gov/ where the nutritional
values of many foods are listed. It would be rather obvious that the names of foods will have to match with
one another

# 2. Implementing the Levenshtein Distance

The Levenshtein distance, also known as the edit distance, is a measure of the similarity between two strings. It is defined as the minimum number of single-character edits (insertions, deletions, or substitutions) required to transform one string into the other.

The Levenshtein distance between two strings can be efficiently computed using a dynamic programming algorithm. Let $s$ and $t$ be the input strings of length $m$ and $n$, respectively. We define a matrix $D$ of size $(m+1) \times (n+1)$, where $D_{i,j}$ represents the Levenshtein distance between the substrings $s_1 \ldots s_i$ and $t_1 \ldots t_j$. The matrix is initialized as follows:

\begin{align}
D_{i,0} &= i, \quad 0 \leq i \leq m \\
D_{0,j} &= j, \quad 0 \leq j \leq n
\end{align}

The matrix is then filled row by row using the following recurrence relation:

\begin{equation}
D_{i,j} = \min \begin{cases}
D_{i-1, j} + 1 \\
D_{i, j-1} + 1 \\
D_{i-1, j-1} + \delta(s_i, t_j)
\end{cases}
\end{equation}

Where $\delta(a, b)$ is an indicator function that returns $0$ if $a = b$ and $1$ otherwise.

After filling the matrix, the Levenshtein distance between strings $s$ and $t$ is given by the value in the bottom-right corner of the matrix:

\begin{equation}
L(s, t) = D_{m,n}
\end{equation}


The file '2019-2020 FNDDS At A Glance - FNDDS Nutrient Values' was slightly altered without some rows and columns into 'stigler nutrional values.csv'. The goal, then will be to pair the most similar strings with the nutritional values:

In [ ]:
import Levenshtein

#  Using the python-Levenshtein library to calculate the Levenshtein distance.

# Read CSV files
file_F = pd.read_csv('ALL_FOOD_PRICE.csv')
file_N = pd.read_csv('stigler nutrional values.csv')


# This code will read the CSV files, compare names using the Levenshtein distance,
# Function to find the most similar name
def find_most_similar_name(name_N, names_F, matched_names):
    best_match = None
    best_match_index = None
    min_distance = float('inf')

    for index, name_F in enumerate(names_F):
        if index not in matched_names:
            distance = Levenshtein.distance(name_N, name_F)

            if distance < min_distance:
                min_distance = distance
                best_match = name_F
                best_match_index = index

    return best_match, best_match_index

# Set to keep track of matched names
# keep track of the names that have already been matched in file
# ensure that a name is not matched more than once.

matched_names = set()

# Iterate through each row in file_N
# If a name in file N has a closer match to a name that has already been matched in file F, 
# it will skip that name and find the next most similar name in file F that hasn't been matched yet.
for index, row in file_N.iterrows():
    name_N = row['Main food description']
    most_similar_name_F, most_similar_name_index = find_most_similar_name(name_N, file_F.iloc[:, 0], matched_names)

    # Update matched_names set
    matched_names.add(most_similar_name_F)

    # Append values from file_N to the corresponding row in file_F
    for col in file_N.columns[1:]:
        file_F.loc[file_F.iloc[:, 0] == most_similar_name_F, col] = row[col]

# Save the updated file_F to a new CSV file
file_F.to_csv('FINALoutput.csv', index=False)

The mostly empty rows (the names that didn't have any matches) are eliminated since nutritional values for them are not available. Additionally, because the nutritional data is in terms of 100 g per unit food, it needs to be converted either into pounds or ounces.

In [ ]:
# Amount per 100 g edible portion for energy and 64 nutrients

# Keep rows with values in all specified columns
specified_columns = list(file_N.columns[1:])
filtered_rows = file_F.dropna(subset=specified_columns)

# Save the filtered rows to a new CSV file
filtered_rows.to_csv('SECONDFINALoutput.csv', index=False)

In [ ]:
# This script reads the SECONDFINALoutput.csv
# file, processes the names in the "Name" column, 
# extracts the numerical value and the 
# letter (L or O), calculates the multiplier, and then 
# multiplies the rest of the row values by 
# the multiplier. The result is saved in a new CSV 
# file called THIRDFINALoutput.csv.

# The first value right after "-" is a 
#numerical value that needs to be converted into a float. 
# The second value after "-"  is a value 
# that starts with either a lower case or upper "L", 
# or a value that starts with  either a lower
#case or upper "O". If the value that starts 
# with either a lower case or upper "L", 
# multiply the first value times 0.220462 to create
# p value, and then multiply the rest of 
# the row values by p. If the value that starts 
# with either a lower case or upper "O", 
# multiply the first value times 3.5274 to create
# h value, and then multiply the rest of 
# the row values by h staring in column 2 and so on

import pandas as pd
import re

def extract_float(text):
    try:
        return float(re.findall('\d+\.\d+|\d+', text)[0])
    except:
        return 0

# Read the CSV file
df = pd.read_csv('SECONDFINALoutput.csv')

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    name = row['Name']
    splitted_name = name.split('-')
    if len(splitted_name) > 1:
        multiplier = extract_float(splitted_name[-1])

        if 'L' in splitted_name[-1].upper():
            multiplier *= 0.220462
        elif 'O' in splitted_name[-1].upper():
            multiplier *= 3.5274

        # Multiply the values in the rest of the columns by the multiplier
        for col in df.columns[2:]:
            df.at[index, col] = df.at[index, col] * multiplier

# Save the updated DataFrame to a new CSV file
df.to_csv('THIRDFINALoutput.csv', index=False)

Now it is possible to use linear optimization to find the Stigler Diet Solution. Sadly, after many trials, a linear optimization algorithm could not be developed y from scratch. Thankfully, the SciPy library has one to finish the project. The scipy.optimize.linprog function source describes the use of the primal-dual path-following method to find the minimal solution in a linear optimization problem.

# 3. The Primal-Dual Path-Following Method

The primal-dual path-following method is an interior-point algorithm for solving linear programming problems. It is an iterative algorithm that seeks a solution to both the primal and the dual linear programming problems simultaneously while maintaining their feasibility and complementary conditions.

Consider the following linear programming problem in the standard form:

\begin{align}
\text{minimize} \quad & \mathbf{c}^T \mathbf{x} \\
\text{subject to} \quad & \mathbf{A} \mathbf{x} = \mathbf{b} \\
& \mathbf{x} \geq \mathbf{0}
\end{align}

Where $\mathbf{x} \in \mathbb{R}^n$ is the decision variable, $\mathbf{c} \in \mathbb{R}^n$ is the cost vector, $\mathbf{A} \in \mathbb{R}^{m \times n}$ is the constraint matrix, and $\mathbf{b} \in \mathbb{R}^m$ is the right-hand side vector.

The dual problem can be formulated as:

\begin{align}
\text{maximize} \quad & \mathbf{b}^T \mathbf{y} \\
\text{subject to} \quad & \mathbf{A}^T \mathbf{y} + \mathbf{s} = \mathbf{c} \\
& \mathbf{s} \geq \mathbf{0}
\end{align}

Where $\mathbf{y} \in \mathbb{R}^m$ is the dual variable and $\mathbf{s} \in \mathbb{R}^n$ is the slack variable.


The primal-dual path-following algorithm maintains a sequence of approximate solutions $(\mathbf{x}^k, \mathbf{y}^k, \mathbf{s}^k)$ to the primal and dual problems. At each iteration, the algorithm seeks to improve the solutions by computing a search direction $(\Delta \mathbf{x}^k, \Delta \mathbf{y}^k, \Delta \mathbf{s}^k)$ that aims to satisfy the feasibility and complementarity conditions:

\begin{align}
\mathbf{A} \mathbf{x}^k &= \mathbf{b} \\
\mathbf{A}^T \mathbf{y}^k + \mathbf{s}^k &= \mathbf{c} \\
\mathbf{x}^k_i \mathbf{s}^k_i &= \mu^k, \quad i = 1, \ldots, n
\end{align}

Where $\mu^k = \frac{\mathbf{x}^{kT} \mathbf{s}^k}{n}$ is the complementarity measure or duality gap, which provides an approximation of how far the current solution is from optimality. The goal of the primal-dual path-following algorithm is to reduce the duality gap while maintaining the feasibility and complementarity conditions.


The search direction $(\Delta \mathbf{x}^k, \Delta \mathbf{y}^k, \Delta \mathbf{s}^k)$ can be computed by solving the following linear system of equations, also known as the affine scaling direction:

\begin{equation}
\begin{bmatrix}
\mathbf{0} & \mathbf{A}^T & \mathbf{I} \\
\mathbf{A} & \mathbf{0} & \mathbf{0} \\
\mathbf{S}^k & \mathbf{0} & \mathbf{X}^k
\end{bmatrix}
\begin{bmatrix}
\Delta \mathbf{x}^k \\
\Delta \mathbf{y}^k \\
\Delta \mathbf{s}^k
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{c} - \mathbf{A}^T \mathbf{y}^k - \mathbf{s}^k \\
\mathbf{b} - \mathbf{A} \mathbf{x}^k \\
-\mathbf{X}^k \mathbf{S}^k \mathbf{e} + \tau \mu^k \mathbf{e}
\end{bmatrix}
\end{equation}

Where $\mathbf{X}^k = \text{diag}(\mathbf{x}^k)$, $\mathbf{S}^k = \text{diag}(\mathbf{s}^k)$, $\mathbf{e}$ is the vector of all ones, and $\tau \in (0, 1)$ is the centering parameter.

After computing the search direction, a line search is performed to find the step sizes $\alpha_p$ and $\alpha_d$ for the primal and dual variables, respectively, such that the new iterates $(\mathbf{x}^{k+1}, \mathbf{y}^{k+1}, \mathbf{s}^{k+1})$ remain feasible:

\begin{align}
\mathbf{x}^{k+1} &= \mathbf{x}^k + \alpha_p \Delta \mathbf{x}^k \\
\mathbf{y}^{k+1} &= \mathbf{y}^k + \alpha_d \Delta \mathbf{y}^k \\
\mathbf{s}^{k+1} &= \mathbf{s}^k + \alpha_d \Delta \mathbf{s}^k
\end{align}

The step sizes are typically chosen such that the iterates do not cross the boundary of the feasible region, and a fraction-to-the-boundary rule is commonly used.

The algorithm terminates when the duality gap $\mathbf{x}^{kT} \mathbf{s}^k$, primal infeasibility $\|\mathbf{A}\mathbf{x}^k - \mathbf{b}\|$, and dual infeasibility $\|\mathbf{A}^T\mathbf{y}^k + \mathbf{s}^k - \mathbf{c}\|$ are smaller than a predefined tolerance level $\epsilon$:

\begin{equation}
\frac{\mathbf{x}^{kT} \mathbf{s}^k}{n} \leq \epsilon \quad \text{and} \quad \|\mathbf{A}\mathbf{x}^k - \mathbf{b}\| \leq \epsilon \quad \text{and} \quad \|\mathbf{A}^T\mathbf{y}^k + \mathbf{s}^k - \mathbf{c}\| \leq \epsilon
\end{equation}

Now we can use linear optimization to find the stigler diet. 

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# Read the CSV file
df = pd.read_csv('THIRDFINALoutput.csv')

# Extract the objective function coefficients 
# (the second column named 'Fourth Column', has 
# the corresponding food prices)
c = df['Fourth Column'].values

# Extract the constraint matrix:
# (nutritional contents: 'Energy (kcal)', 'Protein (g)',
# 'Fatty acids, total monounsaturated and polyunsaturated (good fats) (g)',
# 'Carbohydrate (g)', 'Fiber, total dietary (g)',
 # 'Vitamin A, RAE (mcg_RAE)', 'Vitamin C (mg)', 'Calcium (mg)',
# 'Iron\n(mg)', 'Vitamin D (D2 + D3) (mcg)',
# 'Vitamin E (alpha-tocopherol)  and added (mg)', 'Folate, total (mcg)',
# 'Vitamin B-12, added\n(mcg)')
A_eq = df.iloc[:, 2:].values.T


# Flip the signs of the inequalities to use less-than-or-equal-to constraints
A_ineq = -1 * A_eq

# Constraint bounds (times 7 because we are 
# looking at weekly compsuption, since going 
# every day to the store is time consuming, 
# and buying in bulk may be cheaper)
# I took the upper bound of the minimun requirements 
# for an adult male who is very physically active
b_ineq = np.array([-3000*7, -56*7, -0.2 * 3000*7, -0.45 * 3000*7, -38*7, -700*7, -75*7, -1000*7, -8*7, -600*7, -15*7, -400*7, -2.4*7])

# Solve the problem
res = linprog(c, A_ub=A_ineq, b_ub=b_ineq)

# Print the results
np.set_printoptions(threshold=np.inf)
# print("Optimal solution (Food quantities):", res.x)
print("Minimum cost of the diet:", res.fun)


# Print the food names, their corresponding non-zero quantities, and their prices
food_names = df['Name'].values
food_prices = df['Fourth Column'].values
optimal_quantities = res.x

for name, price, quantity in zip(food_names, food_prices, optimal_quantities):
    if quantity > 1e-6:  # A small threshold to account for floating-point errors
        print(f"{name}: {quantity}, Price: {price}")

Minimum cost of the diet: 20.812124780243167
Lactaid 2% Reduced Fat Milk - 96 Oz: 0.12597395825322255, Price: 7.68
Oatly Oat Milk Original - 64 Oz: 3.445250830010142, Price: 5.76


Looking at the total amount of 7.68 + 5.76 = 13.44, and comparing it with the FDA's report (https://www.fns.usda.gov/cnpp/usda-food-plans-cost-food-reports-monthly-reports) on average weekly food expenditures, which range from 68.70 to 104.70, the expenditures were reduced by a range approximately from 80% to 87%.

This study demonstrates we can approach to solving the Stigler Diet problem by combining modern data scraping techniques, the Levenshtein distance for string matching, and the primal-dual path-following method for linear optimization. By incorporating real-time food prices and comprehensive nutritional information, the proposed solution better reflects the actual dietary needs and cost considerations of individuals. However, it is essential to note that this study's approach is still limited by certain factors, such as missing price information, and most importantly, the limitations of the monotone diet. Future considerations could focus on refining the data collection process, incorporating additional factors like taste preferences and cultural influences, using a condorcet voting system along with the Levenshtein Distance, and extending the method to account for other essential nutrients or non-nutrient factors in a healthy diet.

### References

Levenshtein Distance

https://rosettacode.org/wiki/Levenshtein_distance

http://www.ics.uci.edu/~dan/pubs/p341-hirschberg.pdf

Linear Optimization: The Primal-Dual Path-Following Method

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

https://www.stat.cmu.edu/~ryantibs/convexopt-S15/lectures/16-primal-dual.pdf

https://www.numerical.rl.ac.uk/people/nimg/course/lectures/raphael/lectures/lec16slides.pdf